<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tuning-and-Optimizing-Neural-Networks---Lab" data-toc-modified-id="Tuning-and-Optimizing-Neural-Networks---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tuning and Optimizing Neural Networks - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Loading-the-Data" data-toc-modified-id="Loading-the-Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Loading the Data</a></span></li><li><span><a href="#Defining-the-Problem" data-toc-modified-id="Defining-the-Problem-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Defining the Problem</a></span></li><li><span><a href="#Generating-a-Hold-Out-Test-Set" data-toc-modified-id="Generating-a-Hold-Out-Test-Set-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Generating a Hold Out Test Set</a></span><ul class="toc-item"><li><span><a href="#Preprocessing-Steps" data-toc-modified-id="Preprocessing-Steps-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Preprocessing Steps</a></span></li><li><span><a href="#Preprocess-Your-Holdout-Set" data-toc-modified-id="Preprocess-Your-Holdout-Set-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Preprocess Your Holdout Set</a></span></li></ul></li><li><span><a href="#Defining-a-K-fold-Cross-Validation-Methodology" data-toc-modified-id="Defining-a-K-fold-Cross-Validation-Methodology-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Defining a K-fold Cross Validation Methodology</a></span></li><li><span><a href="#Building-a-Baseline-Model" data-toc-modified-id="Building-a-Baseline-Model-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Building a Baseline Model</a></span></li><li><span><a href="#Evaluating-the-Baseline-Model-with-K-Folds-Cross-Validation" data-toc-modified-id="Evaluating-the-Baseline-Model-with-K-Folds-Cross-Validation-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Evaluating the Baseline Model with K-Folds Cross Validation</a></span></li><li><span><a href="#Intentionally-Overfitting-a-Model" data-toc-modified-id="Intentionally-Overfitting-a-Model-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Intentionally Overfitting a Model</a></span></li><li><span><a href="#Regularizing-the-Model-to-Achieve-Balance" data-toc-modified-id="Regularizing-the-Model-to-Achieve-Balance-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Regularizing the Model to Achieve Balance</a></span></li><li><span><a href="#Final-Evaluation" data-toc-modified-id="Final-Evaluation-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Final Evaluation</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've seen some regularization, initialization and optimization techniques, its time to synthesize those concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but will also attempt to set various hyperparameters for regularization techniques. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Implement a K-folds cross validation modeling pipeline
* Apply normalization as a preprocessing technique
* Apply regularization techniques to improve your model's generalization
* Choose an appropriate optimization strategy 

## Loading the Data

Load and preview the dataset below.

In [3]:
# Your code here; load and preview the dataset
import pandas as pd
data = pd.read_csv("loan_final.csv", header=0)
print(len(data))
data.head()

42538


,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [4]:
# drop rows with no label
data.dropna(subset=['total_pymnt'],inplace=True)
print(len(data))
data.head()

42535


,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42535 entries, 0 to 42537
Data columns (total 16 columns):
loan_amnt              42535 non-null float64
funded_amnt_inv        42535 non-null float64
term                   42535 non-null object
int_rate               42535 non-null object
installment            42535 non-null float64
grade                  42535 non-null object
emp_length             41423 non-null object
home_ownership         42535 non-null object
annual_inc             42531 non-null float64
verification_status    42535 non-null object
loan_status            42535 non-null object
purpose                42535 non-null object
addr_state             42535 non-null object
total_acc              42506 non-null float64
total_pymnt            42535 non-null float64
application_type       42535 non-null object
dtypes: float64(6), object(10)
memory usage: 5.5+ MB


## Defining the Problem

Set up the problem by defining X and y. 

For this problem use the following variables for X:
* loan_amnt
* home_ownership
* funded_amnt_inv
* verification_status
* emp_length
* installment
* annual_inc

Our target variable y will be ```total_pymnt```

In [6]:
# Your code here; appropriately define X and y and apply a train test split
import numpy as np

features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 'home_ownership', 'verification_status', 'emp_length']

X = data.loc[:, features]
y = data.loc[:, 'total_pymnt']

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 10% of the total available data. For consistency of results, use random seed 123. 

In [8]:
# Your code here; generate a hold out test set for final model evaluation. Use random seed 123.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=123)

### Preprocessing Steps
* Fill in missing values with SimpleImputer
* Standardize continuous features with StandardScalar()
* One hot encode categorical features with OneHotEncoder()

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']
X_train_cont = X_train.loc[:, cont_features]

# Fill missing values with the mean
cont_imp = SimpleImputer(strategy='mean')
cont_imp.fit(X_train_cont)
X_train_cont = cont_imp.transform(X_train_cont)

# standardized inputs
sc = StandardScaler()
sc.fit(X_train_cont)
X_train_scaled = sc.transform(X_train_cont)

# Create continuous features dataframe
cont_train_df = pd.DataFrame(X_train_scaled, columns=cont_features)

In [10]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]

# Replace NaNs with 'missing'
cat_imp = SimpleImputer(strategy='constant', fill_value='missing')
cat_imp.fit(X_train_cat)
X_train_cat = cat_imp.transform(X_train_cat)

# Encode Categorical Features
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X_train_cat)
X_train_ohe = ohe.transform(X_train_cat)

# Create categorical features dataframe
cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=ohe.get_feature_names(input_features=cat_features))

In [11]:
# Combine continuous and categorical feature dataframes
X_train_all = pd.concat([cont_train_df, cat_train_df], axis=1)
X_train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38281 entries, 0 to 38280
Data columns (total 24 columns):
loan_amnt                              38281 non-null float64
funded_amnt_inv                        38281 non-null float64
installment                            38281 non-null float64
annual_inc                             38281 non-null float64
home_ownership_MORTGAGE                38281 non-null float64
home_ownership_NONE                    38281 non-null float64
home_ownership_OTHER                   38281 non-null float64
home_ownership_OWN                     38281 non-null float64
home_ownership_RENT                    38281 non-null float64
verification_status_Not Verified       38281 non-null float64
verification_status_Source Verified    38281 non-null float64
verification_status_Verified           38281 non-null float64
emp_length_1 year                      38281 non-null float64
emp_length_10+ years                   38281 non-null float64
emp_length_2 years     

### Preprocess Your Holdout Set

Make sure to use your StandardScalar and OneHotEncoder that you already fit on the training set to transform your test set

In [12]:
# Select continuous features
X_test_cont = X_test.loc[:, cont_features]

# Fill missing values with the mean
X_test_cont = cont_imp.transform(X_test_cont)

# standardized inputs
X_test_scaled = sc.transform(X_test_cont)

# Create continuous features dataframe
cont_test_df = pd.DataFrame(X_test_scaled, columns=cont_features)

In [13]:
# Select only the categorical features
X_test_cat = X_test.loc[:, cat_features]

# Replace NaNs with 'missing'
X_test_cat = cat_imp.transform(X_test_cat)

# Encode Categorical Features
X_test_ohe = ohe.transform(X_test_cat)

# Create categorical features dataframe
cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=ohe.get_feature_names(input_features=cat_features))

In [14]:
# Combine continuous and categorical feature dataframes
X_test_all = pd.concat([cont_test_df, cat_test_df], axis=1)
X_test_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4254 entries, 0 to 4253
Data columns (total 24 columns):
loan_amnt                              4254 non-null float64
funded_amnt_inv                        4254 non-null float64
installment                            4254 non-null float64
annual_inc                             4254 non-null float64
home_ownership_MORTGAGE                4254 non-null float64
home_ownership_NONE                    4254 non-null float64
home_ownership_OTHER                   4254 non-null float64
home_ownership_OWN                     4254 non-null float64
home_ownership_RENT                    4254 non-null float64
verification_status_Not Verified       4254 non-null float64
verification_status_Source Verified    4254 non-null float64
verification_status_Verified           4254 non-null float64
emp_length_1 year                      4254 non-null float64
emp_length_10+ years                   4254 non-null float64
emp_length_2 years                     

## Defining a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, write a function that takes in the remaining data and performs k-folds cross validation given a model object. 

> Note: Think about how you will analyze the output of your models in order to select an optimal model. This may involve graphs, although alternative approaches are certainly feasible.

In [15]:
# Your code here; define a function to evaluate a model object using K folds cross validation.
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
def k_folds(features_train, labels_train, model_obj, k=10, n_epochs=100):
    colors = sns.color_palette("Set2")
    validation_scores = []
    kf = KFold(n_splits=10, shuffle=True)
    fig, axes = plt.subplots(2, 5, figsize=(12,8))
    for i, (train_index, test_index) in enumerate(kf.split(features_train)):
        "Currently graph imaging assumes 10 folds and is hardcoded to 5x2 layout."
        
        row = i//5
        col = i%5
        
        X_train, X_val = features_train.iloc[train_index], features_train.iloc[test_index]
        y_train, y_val = labels_train.iloc[train_index], labels_train.iloc[test_index]
        
        model = model_obj
        
        hist = model.fit(X_train, y_train, batch_size=32,
                         epochs=n_epochs, verbose=0, validation_data = (X_val, y_val))
        #Note: verboxe=0 turns off printouts regarding training for each epoch.
        #Potential simpler methodology
        validation_score = model.evaluate(X_val, y_val)
        validation_scores.append(validation_score)
        ax = axes[row, col]
        k = 'val_loss'
        d = hist.history[k]
        ax.plot(d, label=k, color=colors[0])

        k = 'loss'
        d = hist.history[k]
        ax.plot(d, label=k, color=colors[1])
        ax.set_title('Fold {} Validation'.format(i+1))
        
    #Final Graph Formatting
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)
    plt.legend(bbox_to_anchor=(1,1))
    
    #General Overview
    validation_score = np.average(validation_scores)
    print('Mean Validation Score:', validation_score)
    print('Standard Deviation of Validation Scores:', np.std(validation_scores))
    return validation_score

## Building a Baseline Model

Here, it is also important to define your evaluation metric that you will look to optimize while tuning the model. Additionally, model training to optimize this metric may consist of using a validation and test set if data is plentiful, or k-folds cross-validation if data is limited. Since this dataset is not overly large, it will be most appropriate to set up a k-folds cross-validation  

In [17]:
# Your code here; define and compile an initial model as described
from keras.models import Sequential
from keras.layers import Dense

input_dim = X_train_all.shape[1]

np.random.seed(123)
model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

Using TensorFlow backend.


## Evaluating the Baseline Model with K-Folds Cross Validation

Use your k-folds function to evaluate the baseline model.  

Note: This code block is likely to take 10-20 minutes to run depending on the specs on your computer.
Because of time dependencies, it can be interesting to begin timing these operations for future reference.

Here's a simple little recipe to achieve this:
```
import time
import datetime

now = datetime.datetime.now()
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
```

In [ ]:
# Your code here; use your k-folds function to evaluate the baseline model.
# ⏰ This cell may take several mintes to run
import time
import datetime

now = datetime.datetime.now()

k_folds(X_train_all, y_train, model)

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

3828/3828 [==============================] - 0s 29us/step


## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [ ]:
# Your code here; try some methods to overfit your network
# ⏰ This cell may take several mintes to run
now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

#Model Mod 1: Adding More Layers
model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

k_folds(X_train_all, y_train, model)    

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

In [ ]:
# Your code here; try some methods to overfit your network
# ⏰ This cell may take several mintes to run
now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

k_folds(X_train, y_train, model)    

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

In [ ]:
# Your code here; try some methods to overfit your network
# ⏰ This cell may take several mintes to run
now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

k_folds(X_train_all, y_train, model, n_epochs=250)

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least 2 different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

Notes: 

In [ ]:
# L1 Regularization
from keras import regularizers

#kernel_regularizer=regularizers.l1(0.005)
#kernel_regularizer=regularizers.l2(0.005)
#model.add(layers.Dropout(0.3))

now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

k_folds(X_train_all, y_train, model, n_epochs=250) 

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

In [ ]:
# L2 Regularization and Early Stopping
now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

k_folds(X_train_all, y_train, model, n_epochs=75) 

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

In [ ]:
# Dropout and Early Stopping
from keras import layers

now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(10, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(10, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

k_folds(X_train_all, y_train, model, n_epochs=75) 

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

In [ ]:
# L1, Dropout and Early Stopping

now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

k_folds(X_train_all, y_train, model, n_epochs=75) 

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your finalized model once and for all. Fit the model using all of the training and validation data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)!

In [ ]:
# Your code here; final model training on entire training set followed by evaluation on hold-out data

# Based on our model runs above, it appears that using  L2 Regularization and Early Stopping
# improves our variance 10 fold in exchange for a slight increase in MSE
# As such, we will choose this as our final model in hopes that the model will have improved generalization
now = datetime.datetime.now()

input_dim = X_train_all.shape[1]

model = Sequential()
model.add(Dense(7, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])

hist =  hist = model.fit(X_train_all, y_train, batch_size=32, epochs=75)

later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)

In [ ]:
model.evaluate(X_test_all, y_test)

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline regarding neural networks. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. From here, you'll begin to investigate other neural network architectures such as CNNs.